## Lab 2 - Genetic algorithm

**Autor: Marcin Zwierzchowski**

In [535]:
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.spatial.distance import cdist
import time
from tqdm  import tqdm
import gc
import pandas as pd

## 1. Dane zadania i funkcja pomocnicza

In [442]:
cities = {
    "A": [0, 0],
    "B": [1, 3],
    "C": [2, 1],
    "D": [4, 6],
    "E": [5, 2],
    "F": [6, 5],
    "G": [8, 7],
    "H": [9, 4],
    "I": [10, 8],
    "J": [12, 3]
}
num_to_city = {i+1: city for i, city in enumerate(cities.keys())}
# num_to_city

In [444]:
coordinates = list(cities.values())
distance_matrix = cdist(coordinates, coordinates, metric='euclidean')
# distance_matrix


In [ ]:
def calculate_runtime(algorithm, runs=10, print_output=False):
    measurements = []
    results = {}
    for _ in tqdm(range(runs)):


        gc_old = gc.isenabled()
        gc.disable()
        start_time = time.process_time()
        best_individual, best_score = algorithm()
        end_time = time.process_time()
        if print_output:
            print(f"Best: {[num_to_city[ele] for ele in best_individual]} -> Score: {best_score:.3f}; Route length: {1/best_score:.3f}")
        if gc_old:
            gc.enable()
        measurements.append(end_time - start_time)
    
    results = {
        "Min": round(np.min(measurements), 2),
        "Avg": round(np.mean(measurements), 2),
        "Std": round(np.std(measurements), 2),
        "Max": round(np.max(measurements), 2)
    }

    return pd.DataFrame([results])

## 2. Implementacja algorytmu genetycznego

In [ ]:
class GeneticAlgorithm:
    """
    Genetic Algorithm for solving the Traveling Salesman Problem (TSP).

    Parameters:
    - distance_matrix: 2D array representing distances between cities.
    - tmax (int): Maximum number of iterations (default: 50000).
    - Pm (float): Mutation probability (default: 0.6).
    - Pc (float): Crossover probability (default: 0.6).
    - pop_size (int): Population size (default: 5).
    - num_cities (int): Number of cities (default: 10).
    - selection_method (str): Selection method ('roulette' or 'tournament', default: 'roulette').
    """

    def __init__(self, distance_matrix, 
                 tmax = 1000, 
                 Pm = 0.1, Pc = 0.8, 
                 pop_size = 50, 
                 selection_method = "roulette",
                 verbose = False):
        
        self.tmax = tmax
        self.population = []
        self.Pm = Pm
        self.Pc = Pc
        self.pop_size = pop_size
        self.num_cities = distance_matrix.shape[0]
        self.distance_matrix = distance_matrix
        self.selection_method = selection_method
        self.verbose = verbose
        self._validate_parameters()

    def _validate_parameters(self):
    
        if not (0 <= self.Pm <= 1):
            raise ValueError("Mutation probability (Pm) must be between 0 and 1.")
        if not (0 <= self.Pc <= 1):
            raise ValueError("Crossover probability (Pc) must be between 0 and 1.")
        if self.pop_size <= 0:
            raise ValueError("Population size (pop_size) must be greater than 0.")
        if not isinstance(self.distance_matrix, (list, np.ndarray)):
            raise TypeError("Distance matrix must be a list or numpy array.")
        if self.selection_method not in ["roulette", "tournament"]:
            raise ValueError("Selection method must be 'roulette' or 'tournament'.")  
        if self.distance_matrix.shape[0] == 0:
            raise ValueError("Distance matrix must contain at least one city.") 
        if self.distance_matrix.shape[0] != self.distance_matrix.shape[1]:
            raise ValueError("Distance matrix must be square (same number of rows and columns).") 

    def initialize_population(self):

        for _ in range(self.pop_size):
            individual = random.sample(range(1,self.num_cities+1), self.num_cities)
            individual.append(individual[0])  
            self.population.append(individual)


    def run(self):

        t = 0 
        fitness_scores = self._evaluate_population(self.population) 
        best_score = max(fitness_scores)
        best_individual = self.population[fitness_scores.index(best_score)].copy()
        

        while(t < self.tmax):
            if self.verbose:
                print(f"------------------Iteration: {t+1}/{self.tmax}-------------------")
                print(f"Best individual: {best_individual} -> {best_score:.4f}. Route length: {1/best_score:.3f}")
                # for individual in self.population:
                #     print(individual)
            

            
            if self.selection_method == "tournament":
                reproduction_pop = self._tournament_selection(self.population, fitness_scores)
            elif self.selection_method == "roulette":
                reproduction_pop = self._roulette_wheel_selection(self.population, fitness_scores) 

            crossovered_pop = self._crossover_population(reproduction_pop) 
            mutants_pop = self._mutation(crossovered_pop)
             

            fitness_scores = self._evaluate_population(mutants_pop)
            score = max(fitness_scores)
            individual = mutants_pop[fitness_scores.index(score)]
            
            if score > best_score:
                best_score = score
                best_individual = individual
            
            self.population = mutants_pop
            t+=1
        return best_individual, float(best_score)
    

    def _fitness_fun(self,individual):
        
        total_dist = 0
        for n, _ in enumerate(individual[:-1]):
            total_dist += self.distance_matrix[individual[n]-1, individual[n+1]-1]
        return 1/total_dist
    

    def _evaluate_population(self, population):

        fitness_scores = []
        for individual in population:
            score = self._fitness_fun(individual)
            fitness_scores.append(score)
        return fitness_scores


    def _roulette_wheel_selection(self, population, fitness_scores):
        
        reproduction_pop = []
        total_fitness = sum(fitness_scores)

        for _ in range(self.pop_size): 
            pick = random.uniform(0, total_fitness)
            current = 0

            # Selection of an individual based on probability. Individuals with higher fitness values have a greater chance of being selected.
            for i, fitness in enumerate(fitness_scores):
                current += fitness
                if pick <= current: 
                    # print(f"Selected individual no. {i}: {population[i]} -> {current - fitness :.1f} < {pick:.1f} < {current:.1f}")
                    reproduction_pop.append(population[i])
                    break
        return reproduction_pop


    def _tournament_selection(self, population, fitness_scores):
        
        reproduction_pop = []
        for _ in range(self.pop_size): 
            pick1 = random.randint(0, self.pop_size-1)
            pick2 = random.randint(0, self.pop_size-1)
            
            # print(f" {len(fitness_scores)} Selected individual numbers: {pick1}  {pick2}")

            if fitness_scores[pick1] > fitness_scores[pick2]:
                reproduction_pop.append(population[pick1])
            else:
                reproduction_pop.append(population[pick2])
        return reproduction_pop
        

    def _one_point_crossover(self, parent1, parent2):
        
        point = random.randint(1, len(parent1) - 2) 
        # Ensure the crossover point is not the first and not penultimate element or last one, 
        # because then it would give same parents.

        # print(f"Selected crossover point: {point}")

        child1 = parent1[:point]
        for gene in parent2:
            if gene not in child1:
                child1.append(gene)
        child1.append(child1[0])  

        child2 = parent2[:point]
        for gene in parent1:
            if gene not in child2:
                child2.append(gene)
        child2.append(child2[0])  

        return child1, child2
    

    def _crossover_population(self, population):

        new_population = []
        random.shuffle(population)

        for i in range(0, len(population) - 1, 2):  # Process pairs (0-1, 2-3, ...)
            parent1, parent2 = population[i], population[i + 1]
            
            if random.random() < self.Pc:  
                child1, child2 = self._one_point_crossover(parent1, parent2)
                new_population.extend([child1, child2])
            else:
                new_population.extend([parent1.copy(), parent2.copy()]) 

        # Case where the population size is odd
        if len(population) % 2 == 1:
            new_population.append(population[-1].copy())   

        return new_population


    def _mutation(self, population):
        mutated_population = []
        for individual in population:
            if random.random() < self.Pm:
                # print(f"Mutating individual: {individual}")
                mutated_population.append(self._swap_mutation(individual))
            else:
                mutated_population.append(individual)
        return mutated_population
    
    def _swap_mutation(self, route):
        #Random swap of two cities in the route, except for the first and last
        mutated_route = route[:]
        i, j = random.sample(range(1, len(route) - 1), 2)  
        mutated_route[i], mutated_route[j] = mutated_route[j], mutated_route[i]  
        assert mutated_route[0] == route[0] and mutated_route[-1] == route[-1]

        return mutated_route
    

## 3. Przykładowe użycie

### 3.1 Selekcja ruletkowa

In [579]:
runs = 10
tmax = 300

In [ ]:
genetic_algorithm = GeneticAlgorithm(distance_matrix, 
                tmax = tmax, 
                Pm = 0.1, Pc = 0.8, 
                pop_size = 500, 
                selection_method = "roulette")
genetic_algorithm.initialize_population()

In [581]:
runtime = calculate_runtime(genetic_algorithm.run, runs=runs, print_output=True)
print(f"Runtime")
print(runtime)

 10%|█         | 1/10 [00:08<01:18,  8.76s/it]

Best: ['F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F'] -> Score: 0.030; Route length: 33.123


 20%|██        | 2/10 [00:17<01:08,  8.59s/it]

Best: ['J', 'I', 'G', 'F', 'D', 'B', 'A', 'C', 'E', 'H', 'J'] -> Score: 0.030; Route length: 33.123


 30%|███       | 3/10 [00:25<00:59,  8.56s/it]

...

Best: ['F', 'H', 'J', 'I', 'G', 'D', 'B', 'C', 'A', 'E', 'F'] -> Score: 0.028; Route length: 35.331


 90%|█████████ | 9/10 [01:03<00:06,  6.34s/it]

Best: ['F', 'D', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'F'] -> Score: 0.028; Route length: 35.561


100%|██████████| 10/10 [01:09<00:00,  6.96s/it]

Best: ['F', 'D', 'B', 'A', 'C', 'E', 'H', 'J', 'I', 'G', 'F'] -> Score: 0.030; Route length: 33.123
Runtime
    Min   Avg   Std   Max
0  5.88  6.78  0.85  8.05


### 3.2 Selekcja turniejowa

In [ ]:
genetic_algorithm = GeneticAlgorithm(distance_matrix, 
                tmax = tmax, 
                Pm = 0.1, Pc = 0.8, 
                pop_size = 500, 
                selection_method = "tournament")
genetic_algorithm.initialize_population()

In [583]:
runtime = calculate_runtime(genetic_algorithm.run, runs=runs, print_output=True)
print(f"Runtime")
print(runtime)

 10%|█         | 1/10 [00:01<00:09,  1.09s/it]

Best: ['B', 'D', 'F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B'] -> Score: 0.030; Route length: 33.123


 20%|██        | 2/10 [00:02<00:08,  1.06s/it]

Best: ['F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F'] -> Score: 0.030; Route length: 33.123


 30%|███       | 3/10 [00:03<00:07,  1.06s/it]

...

Best: ['F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F'] -> Score: 0.030; Route length: 33.123


 90%|█████████ | 9/10 [00:09<00:01,  1.09s/it]

Best: ['F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F'] -> Score: 0.030; Route length: 33.123


100%|██████████| 10/10 [00:11<00:00,  1.11s/it]

Best: ['F', 'G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F'] -> Score: 0.030; Route length: 33.123
Runtime
    Min  Avg   Std   Max
0  1.03  1.1  0.07  1.28


### 3.2 Modyfikacja Pm

#### 3.2.1 Selekcja ruletkowa

In [584]:
genetic_algorithm = GeneticAlgorithm(distance_matrix, 
                tmax = tmax, 
                Pm = 0.6, Pc = 0.8, 
                pop_size = 500, 
                selection_method = "roulette")
genetic_algorithm.initialize_population()

runtime = calculate_runtime(genetic_algorithm.run, runs=runs, print_output=True)
print(f"Runtime")
print(runtime)

 10%|█         | 1/10 [00:07<01:08,  7.65s/it]

Best: ['A', 'C', 'E', 'F', 'H', 'J', 'I', 'G', 'D', 'B', 'A'] -> Score: 0.029; Route length: 34.034


 20%|██        | 2/10 [00:14<00:56,  7.09s/it]

Best: ['G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F', 'G'] -> Score: 0.030; Route length: 33.123


 30%|███       | 3/10 [00:21<00:48,  6.97s/it]

...

Best: ['G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F', 'G'] -> Score: 0.030; Route length: 33.123


 90%|█████████ | 9/10 [01:03<00:07,  7.25s/it]

Best: ['G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F', 'G'] -> Score: 0.030; Route length: 33.123


100%|██████████| 10/10 [01:10<00:00,  7.05s/it]

Best: ['G', 'I', 'J', 'H', 'E', 'C', 'A', 'B', 'D', 'F', 'G'] -> Score: 0.030; Route length: 33.123
Runtime
    Min  Avg   Std   Max
0  6.64  7.0  0.37  7.55


#### 3.2.2 Selekcja turniejowa

In [585]:
genetic_algorithm = GeneticAlgorithm(distance_matrix, 
                tmax = tmax, 
                Pm = 0.6, Pc = 0.8, 
                pop_size = 500, 
                selection_method = "tournament")
genetic_algorithm.initialize_population()

runtime = calculate_runtime(genetic_algorithm.run, runs=runs, print_output=True)
print(f"Runtime")
print(runtime)

 10%|█         | 1/10 [00:01<00:11,  1.27s/it]

Best: ['E', 'C', 'A', 'B', 'D', 'F', 'G', 'I', 'J', 'H', 'E'] -> Score: 0.030; Route length: 33.123


 20%|██        | 2/10 [00:02<00:10,  1.29s/it]

Best: ['E', 'H', 'J', 'I', 'G', 'F', 'D', 'B', 'A', 'C', 'E'] -> Score: 0.030; Route length: 33.123


 30%|███       | 3/10 [00:04<00:09,  1.38s/it]

...

Best: ['E', 'C', 'A', 'B', 'D', 'F', 'G', 'I', 'J', 'H', 'E'] -> Score: 0.030; Route length: 33.123


 90%|█████████ | 9/10 [00:13<00:01,  1.52s/it]

Best: ['E', 'C', 'A', 'B', 'D', 'F', 'G', 'I', 'J', 'H', 'E'] -> Score: 0.030; Route length: 33.123


100%|██████████| 10/10 [00:14<00:00,  1.43s/it]

Best: ['E', 'C', 'A', 'B', 'D', 'F', 'G', 'I', 'J', 'H', 'E'] -> Score: 0.030; Route length: 33.123
Runtime
    Min   Avg   Std   Max
0  1.28  1.43  0.13  1.67


## 4. Results

| Selection| Pm       | Min      | Avg      | Std      | Max      | Exact solution |
|----------|----------|----------|----------|----------|----------|----------------|
| Ruletkowa| 0.1      | 4s       | 4.82s    | 0.58s    | 6.3s     |       3/10     |
|Turniejowa| 0.1      | 1.03s    | 1.1s     | 0.07s    | 1.28s    |       10/10    |
| Ruletkowa| 0.6      | 6.64s    | 7s       | 0.37s    | 7.55s    |       9/10     |
|Turniejowa| 0.6      | 1.28s    | 1.43s    | 0.13s    | 1.67s    |      10/10     |

Algorytm genetyczny zaimplementowany wraz z selekcja turniejowa jest szybsza oraz daje lepsze wyniki. W pierwszym przypadku 
algorytm wraz z selekcją ruletkową dał dokładne rozwiązanie 3 z 10 razy, gdzie algorytm wraz z selekcją turniejową (inne parametry takie same) dał 10/10 dokładnych rozwiązań i był przy tym szybszy średnio o 3s. Podobnie selekcja turniejowa ma przewagę dla większego prawdopodobieństwa mutacji = 0.6. Tu była szybsza o ponad 5s i dała lepsze wyniki, choć już nie tak znacząco. 
Zwiększanie prawdopodobieństwa mutacji zwiększa czas działania algorytmu, ale i poprawia szanse na osiągnięcia minimum globalnego. Widać to na przykładzie selekcji ruletkowej, która przy Pm = 0.1 znajdywała 3/10 minimum globalne, a dla Pm=0.6 już 10/10. Dzieje się to kosztem czasu działania algorytmu. 
